In [153]:
!pip install simpletransformers

In [154]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from simpletransformers.ner import NERModel,NERArgs
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score

In [156]:
data=pd.read_csv("/kaggle/input/maindataset/NER dataset1.csv",encoding="latin-1")
df=pd.DataFrame(data)
display(df)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [157]:
data.head(20)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [158]:
data.isna().sum()

Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64

In [159]:
data=data.fillna(method="ffill")
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [160]:
data.columns

Index(['Sentence #', 'Word', 'POS', 'Tag'], dtype='object')

In [161]:
row=data.columns[0]
row

'Sentence #'

In [162]:
data.rename({row:"sentence_id",'Word':"words","POS":"pos","Tag":"label"},axis=1,inplace=True)
data

,sentence_id,words,pos,label
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [163]:
e=data.loc[(data["label"]=="I-per") | (data["label"]=="B-per")] # filter the human name only
e.head(50)

,sentence_id,words,pos,label
42,Sentence: 2,Bush,NNP,B-per
270,Sentence: 13,Daynesha Glover,NNP,B-per
271,Sentence: 13,Mahmoud,NNP,I-per
272,Sentence: 13,Ahmadinejad,NNP,I-per
331,Sentence: 15,Thomas,NNP,B-per
332,Sentence: 15,Horbach,NNP,I-per
443,Sentence: 19,Daynesha Glover,NNP,B-per
444,Sentence: 19,Abdullahi,NNP,I-per
445,Sentence: 19,Yusuf,NNP,I-per
446,Sentence: 19,Ahmad,NNP,I-per


In [164]:
enc=LabelEncoder()
data["sentence_id"]=enc.fit_transform(data["sentence_id"])
display(data.head(30))

,sentence_id,words,pos,label
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [165]:
data["label"]=data["label"].str.upper()
data.columns

Index(['sentence_id', 'words', 'pos', 'label'], dtype='object')

In [166]:
x=data[["sentence_id","words"]]
y=data["label"]

In [167]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [168]:
train_data=pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data=pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [169]:
train_data

,sentence_id,words,labels
592908,19009,and,O
184386,46224,in,O
1021782,40810,in,O
59606,18823,ruling,O
93792,36357,law,O
...,...,...,...
963395,37825,in,O
117952,42818,.,O
435829,10999,The,O
305711,4462,in,O


## MODEL TRANING

In [170]:
label=data["label"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [171]:
args=NERArgs()
args.num_train_epochs=1
args.learning_rate=1e-4
args.overwrite_output_dir=True
args.train_batch_size=32
args.eval_batch_size=32
args.optimizer="AdamW"

In [172]:
model=NERModel(model_type="bert", model_name="bert-base-cased",labels=label,args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

In [ ]:
model.train_model(train_data=train_data,eval_data=test_data,acc=accuracy_score)

/opt/conda/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_data)

In [ ]:
result

In [ ]:
predictions, raw_outputs = model.predict(["Nyunt and  going to School Azahari,Vignesh and Yusuf are friends Nancy-Amelia and Pooja playing footbal. Raja homedown in Madurai"])

In [ ]:
predictions

## Deployement

In [ ]:
import pickle as pkl
pkl.dump(model,open("NERBE.pkl","wb"))

In [ ]:
read=pkl.load(open("NERBE.pkl","rb"))
read

In [ ]:
predictions, raw_outputs = read.predict(["Nyunt and Rodney and  Keveoni Simmons Balser going to School Azahari and Vignesh and Yusuf are friends Kimberly Barnes and Pooja playing footbal. Raja homedown in Madurai"])

In [ ]:
predictions

In [ ]:
filtered_names = []
for tokens in predictions[0]:
    for name, label in tokens.items():
        if label == 'I-PER' or label == 'B-PER':
            filtered_names.append(name)

print(filtered_names)

In [ ]:
!pip install pdfminer.six


In [ ]:
import re
from pdfminer.high_level import extract_text,extract_pages,extract_text_to_fp
def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text
text = extract_text('/kaggle/input/lease6/Lease (6).pdf')
cleaned_text = remove_punctuation(text)

In [ ]:
#cleaned_text 

In [ ]:
result, raw_outputs = read.predict([cleaned_text])

In [ ]:
filtered_name = []
for tokens in result[0]:
    for name, label in tokens.items():
        if label == 'I-PER' or label == 'B-PER':
            filtered_name.append(name)

print(filtered_name)

In [ ]:
filtered_name

In [ ]:
data = {}
data['name_count'] = len(filtered_name)
data['names'] = [{"value": name} for name in filtered_name]

In [ ]:
data

## Tag features

* O: Outside (no named entity)
* B-geo: Beginning of a geographical entity
* B-tim: Beginning of a time entity
* B-org: Beginning of an organization entity
* I-per: Inside of a person's name entity
* B-per: Beginning of a person's name entity
* I-org: Inside of an organization entity
* B-gpe: Beginning of a geopolitical entity
* I-geo: Inside of a geographical entity
* I-tim: Inside of a time entity
* B-art: Beginning of an artifact entity
* B-eve: Beginning of an event entity
* I-art: Inside of an artifact entity
* I-eve: Inside of an event entity
* B-nat: Beginning of a natural phenomenon entity
* I-gpe: Inside of a geopolitical entity
* I-nat: Inside of a natural phenomenon entity